## Preparación de los datos.

El proposito de este notebook es recolectar los datasets que se utilizaran en este estudio y generar un dataset limpio para realizar analisis.

In [185]:
# Importamos las librerias para el manejo de datasets
import pandas as pd
import numpy as np
import os

### Métodos preparación datos.

In [186]:
MODE_RUN = 'PRD' # Modo de ejecucion del notebook
URL_DATA = 'Data/' # Ruta de data sin procesar 
URL_PREPARED_DATA = 'Prepared_Data/' # Ruta de data procesada

In [187]:
def printmode(value, mode=MODE_RUN):
    if mode == 'DEV':
        print(value)

In [188]:
def uppercasecolumns(dataframe):
    for column in dataframe.columns:
        dataframe.rename(columns = {column:column.upper()}, inplace = True)

In [189]:
def createprimarykey(dataframe):
    lskeys = []
    for index, row in dataframe.iterrows():
        fecha = row['FECHA']
        fechaDate = fecha.to_pydatetime()
        keyid = str(row['IDFORMULARIO']) + str(fechaDate.day) + str(fechaDate.month) + str(fechaDate.year)
        lskeys.append(int(keyid))
    return dataframe.insert(0,"KEYID", lskeys)

In [190]:
def renamecolumn(dataframe,columnName,columnNameChanged):
    dataframe.rename(columns= {columnName: columnNameChanged.upper() }, inplace = True)

In [191]:
def deletecolumnsbyiterable(dataframe,columns):
    for column in columns:
        del dataframe[column]
    return dataframe

### Extracción de datasets a dataframe.

In [192]:
sina = pd.read_excel(URL_DATA+'Base_2017.xlsx',sheet_name ='ACCIDENTES')

In [193]:
sinc = pd.read_excel(URL_DATA+'Base_2017.xlsx',sheet_name ='CONDUCTORES')

In [194]:
sinv = pd.read_excel(URL_DATA+'Base_2017.xlsx',sheet_name ='VICTIMAS')

### Conversión  nombres columnas por dataframe.

In [195]:
# Todos los nombres de las columnas se volveran UPPERCASE
uppercasecolumns(sina)
uppercasecolumns(sinc)
uppercasecolumns(sinv)

Se renombra el campo ID de los dataframes del 2017 para tener un mismo formato para la creacion de las primary keys de los cada dataframe

In [196]:
renamecolumn(sina,'ID','IDFORMULARIO')
renamecolumn(sinc,'ID','IDFORMULARIO')
renamecolumn(sinv,'ID','IDFORMULARIO')

In [197]:
printmode('------------------------ ACCIDENTES 2017 ------------------------')
printmode(sina.columns)

In [198]:
printmode('------------------------ CONDUCTORES 2017 ------------------------')
printmode(sinc.columns)

In [199]:
printmode('------------------------ VICTIMAS 2017 ------------------------')
printmode(sinv.columns)

In [200]:
printmode(sina.shape)

### Creación primary keyId por dataframe.

In [201]:
createprimarykey(sina)

In [202]:
createprimarykey(sinc)

In [203]:
createprimarykey(sinv)

In [204]:
printmode('------------------------ ACCIDENTES 2017 ------------------------')
printmode(sina.columns)

In [205]:
printmode('------------------------ CONDUCTORES 2017 ------------------------')
printmode(sinc.columns)

In [206]:
printmode('------------------------ VICTIMAS 2017 ------------------------')
printmode(sinv.columns)

### Validación columnas por dataFrame.

Se debe agregar un datframe principal el cual se valide las columnas que debe detener los dataframe nuevos, de no tener las columnas del principal se debene agregar las que falten y eliminar las que sobren., 

### Elimina columnas no existentes por dataframe.

In [207]:
sina = sina.copy()
printmode(sina.shape)

In [208]:
sinc = sinc.copy()
printmode(sinc.shape)

In [209]:
sinv = sinv.copy()
printmode(sinv.shape)

In [210]:
def searchcolumns(dataframe, dfcomparates):
    setdataframe = set()
    for column in dataframe.columns:
         if column not in dfcomparates:
            setdataframe.add(column)
    return setdataframe

### Columnas a conservar

In [211]:
colacci = set (['KEYID','GRAVEDADCOD','DIRECCION','HORA_PROCESADA','LOCALIDAD'])

In [212]:
colcond = set (['LLEVACASCO', 'LLEVACINTURON', 'CON_EMBRIAGUEZ', 'GRAVEDAD_PROCESADA', 'POSSESEGURORESPONSABILIDAD',
	'CON_HUECOS', 'SEXO', 'LLEVACHALECO', 'MODELOVEHICULO', 'SERVICIOVEHICULO', 'CLASEVEHICULO', 
	'DIA_PROCESADO', 'CON_PEATON', 'FECHA', 'KEYID', 'EDAD_PROCESADA'])

In [213]:
colvict = set (['KEYID'])

### Columnas a eliminar

In [214]:
setcolcond = searchcolumns(sinc.copy(),colcond)
printmode(setcolcond)

In [215]:
setcolacci = searchcolumns(sina.copy(),colacci)
printmode(setcolacci)

In [216]:
setcolvict = searchcolumns(sinv.copy(),colvict)
printmode(setcolvict)

In [217]:
def delete_columns_structuring(dataframe, columns):
    for column in columns:
        if column in dataframe.columns:
            del dataframe[column]
    return dataframe

### Eliminacion de columnas

In [218]:
dfsina = delete_columns_structuring(sina.copy(),setcolacci)

In [219]:
dfsinc = delete_columns_structuring(sinc.copy(),setcolcond)

In [220]:
dfsinv = delete_columns_structuring(sinv.copy(),setcolvict)

In [221]:
printmode(len(colacci)), printmode(dfsina.shape)

(None, None)

In [222]:
printmode(len(colcond)), printmode(dfsinc.shape)

(None, None)

In [223]:
printmode(len(colvict)), printmode(dfsinv.shape)

(None, None)

In [224]:
siniaccichild=dfsina.copy()

In [225]:
sinicond=dfsinc.copy()

### Se obtienen columnas de accidentes para conductores

In [226]:
sinicond.insert(len(sinicond.columns),'GRAVEDADCOD',[siniaccichild[siniaccichild['KEYID']==row['KEYID']]['GRAVEDADCOD'].values[0] for index,row in sinicond.iterrows()])

In [227]:
sinicond.insert(len(sinicond.columns),'DIRECCION',[siniaccichild[siniaccichild['KEYID']==row['KEYID']]['DIRECCION'].values[0] for index,row in sinicond.iterrows()])

In [228]:
sinicond.insert(len(sinicond.columns),'LOCALIDAD',[siniaccichild[siniaccichild['KEYID']==row['KEYID']]['LOCALIDAD'].values[0] for index,row in sinicond.iterrows()])

In [229]:
sinicond.insert(len(sinicond.columns),'HORA_PROCESADA',[siniaccichild[siniaccichild['KEYID']==row['KEYID']]['HORA_PROCESADA'].values[0] for index,row in sinicond.iterrows()])

Inicio a las 9:09 - 9:12 Toma aprox 3 - 4 min 

### Se exporta el dataframe 

In [230]:
sinicond.to_csv(URL_PREPARED_DATA+'siniestros-prepared-1.csv')

Inicio 9:21 - 9:25 Toma aprox 3-4 min